# [엑셀 => 데이터프레임 저장 다루기]


(1) 모듈 로딩

In [2]:
import pandas as pd


(2) 데이터 준비

In [3]:
filename = '../DATA/Single Worksheet.xlsx'
filename2 = '../DATA/Multiple Worksheets.xlsx'

(3) Excel => DataFrame 저장하기

In [4]:
pd.read_excel(filename)

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [5]:
pd.read_excel(filename, header=None, usecols=[1,3])

,1,3
0,Last Name,Gender
1,James,M
2,Hawkins,M
3,Day,F
4,Ruiz,F
5,Gomez,F


In [6]:
pd.read_excel(filename2, sheet_name='Data 1')

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


## [JSON ==> DataFrame 저장 ]

(1) Prepare Data and Load it

In [7]:
nobel_DF = pd.read_json('../DATA/nobel.json')

In [8]:
nobel_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646 entries, 0 to 645
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prizes  646 non-null    object
dtypes: object(1)
memory usage: 10.1+ KB


(2) Check Data

In [9]:
nobel_DF.head(3), nobel_DF.tail(3)

(                                              prizes
 0  {'year': '2019', 'category': 'chemistry', 'lau...
 1  {'year': '2019', 'category': 'economics', 'lau...
 2  {'year': '2019', 'category': 'literature', 'la...,
                                                 prizes
 643  {'year': '1901', 'category': 'peace', 'laureat...
 644  {'year': '1901', 'category': 'physics', 'laure...
 645  {'year': '1901', 'category': 'medicine', 'laur...)

(3) extract data

In [10]:
data2019 = nobel_DF.iloc[0]['prizes']
type(data2019)

dict

In [11]:
for k, v in data2019.items():
    if type(v) == type(list()):
        for ddd in v:
            for kk, vv in ddd.items():
                print(f"[{kk}]\n ====> {vv}")
    else:
        print(f"[{k}]\n ====> {v}")
        

[year]
 ====> 2019
[category]
 ====> chemistry
[id]
 ====> 976
[firstname]
 ====> John
[surname]
 ====> Goodenough
[motivation]
 ====> "for the development of lithium-ion batteries"
[share]
 ====> 3
[id]
 ====> 977
[firstname]
 ====> M. Stanley
[surname]
 ====> Whittingham
[motivation]
 ====> "for the development of lithium-ion batteries"
[share]
 ====> 3
[id]
 ====> 978
[firstname]
 ====> Akira
[surname]
 ====> Yoshino
[motivation]
 ====> "for the development of lithium-ion batteries"
[share]
 ====> 3


- json 중첩 데이터의 다중 반복문 처리 ==> 달일 리스트 형태 처리 : 평탄화 / 정규화
- json_normalize()

In [12]:
pd.json_normalize(data2019, record_path='laureates', meta=['year', 'category']).set_index('id')

,firstname,surname,motivation,share,year,category
id,,,,,,
976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


- 모든 행에 대한 정규화/평탄화 진행하기

In [13]:
# 컬럼 읽기 => DF객체변수명['컬럼명'], DF객체변수명.컼컮명
nobel_DF.prizes

0      {'year': '2019', 'category': 'chemistry', 'lau...
1      {'year': '2019', 'category': 'economics', 'lau...
2      {'year': '2019', 'category': 'literature', 'la...
3      {'year': '2019', 'category': 'peace', 'laureat...
4      {'year': '2019', 'category': 'physics', 'overa...
                             ...                        
641    {'year': '1901', 'category': 'chemistry', 'lau...
642    {'year': '1901', 'category': 'literature', 'la...
643    {'year': '1901', 'category': 'peace', 'laureat...
644    {'year': '1901', 'category': 'physics', 'laure...
645    {'year': '1901', 'category': 'medicine', 'laur...
Name: prizes, Length: 646, dtype: object

In [24]:
# 모든 행의 record_path 에 해당하는 키(key)가 존제해야함
# 따라서 없는 행에 record_path 에 해당하는 키를 추가해야함
# pd.json_normalize(nobel_DF.prizes, record_path='laureates', meta=['year', 'category'])

for data in nobel_DF.prizes:
    # if 'laureates' not in data.keys():
    #     data['laureates'] = []
    data.setdefault('laureates', [])
        

In [26]:
nobel_DF.prizes
nobel_normalized = pd.json_normalize(nobel_DF.prizes, record_path='laureates', meta=['year', 'category'])

In [27]:
nobel_normalized

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry
3,982,Abhijit,Banerjee,"""for their experimental approach to alleviatin...",3,2019,economics
4,983,Esther,Duflo,"""for their experimental approach to alleviatin...",3,2019,economics
...,...,...,...,...,...,...,...
945,569,Sully,Prudhomme,"""in special recognition of his poetic composit...",1,1901,literature
946,462,Henry,Dunant,"""for his humanitarian efforts to help wounded ...",2,1901,peace
947,463,Frédéric,Passy,"""for his lifelong work for international peace...",2,1901,peace
948,1,Wilhelm Conrad,Röntgen,"""in recognition of the extraordinary services ...",1,1901,physics


In [29]:
# nobel_DF.apply(lambda x:x.setdefault('laureates', []), axis=0)

AttributeError: 'Series' object has no attribute 'setdefault'

- (7) JSON의 다양한 방식의 DataFrame 저장하기 => orient 파라미터

In [34]:
# 데이터 => 키 : {값}
json_data = '''
[
    { "id": "1", "name": "test1", "age": 22},
    { "id": "2", "name": "test2", "age": 23},
    { "id": "3", "name": "test3", "age": 24},
    { "id": "4", "name": "test4", "age": 25}
]'''


- orient = "record" 설정 => JSON 파일 데이터를 행 단위로 읽어서 DataFrame에 저장

In [35]:
df = pd.read_json(json_data, orient='records')
df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17180\2704247045.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data, orient='records')


,id,name,age
0,1,test1,22
1,2,test2,23
2,3,test3,24
3,4,test4,25


In [37]:
json_data2 = '''
{
    "std1": {"name":"가나다", "age": 22},
    "std2": {"name":"가나다", "age": 22},
    "std3": {"name":"가나다", "age": 22},
    "std4": {"name":"가나다", "age": 22}
}
'''

df2 = pd.read_json(json_data2, orient='index')
df2

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17180\688710320.py:10: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_json(json_data2, orient='index')


,name,age
std1,가나다,22
std2,가나다,22
std3,가나다,22
std4,가나다,22


In [41]:
df22 = pd.read_json(json_data2, orient='columns')
df22

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17180\805640869.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df22 = pd.read_json(json_data2, orient='columns')


,std1,std2,std3,std4
name,가나다,가나다,가나다,가나다
age,22,22,22,22


In [44]:
json_data3 = '''
[
    ["1","가나다",33],
    ["1","가나다",33],
    ["1","가나다",33],
    ["1","가나다",33],
    ["1","가나다",33],
    ["1","가나다",33],
    ["1","가나다",33]
]
'''
pd.read_json(json_data3, orient='values')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17180\2754054162.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(json_data3, orient='values')


,0,1,2
0,1,가나다,33
1,1,가나다,33
2,1,가나다,33
3,1,가나다,33
4,1,가나다,33
5,1,가나다,33
6,1,가나다,33
